In [1]:
from openforcefield.topology import Molecule
from openforcefield.typing.engines.smirnoff import ForceField
from simtk.openmm.app import PDBFile 
from openforcefield.topology import Topology
from openforcefield.typing.engines.smirnoff import ForceField
from simtk.openmm import XmlSerializer
from pprint import pprint

Unable to load toolkit <openforcefield.utils.toolkits.OpenEyeToolkitWrapper object at 0x116f3b438>.


## Parametrising Ethane with SMIRNOFF and running it with BioSimSpace

In [2]:
#mol = Molecule.from_smiles('CC')
lig = Molecule.from_file('ethane.sdf')

In [3]:
## needed for atom naming
for idx, atom in enumerate(lig.atoms):
    atom.name = f'{atom.element.symbol}{idx}'
topology = Topology.from_molecules(lig)

In [4]:
forcefield = ForceField('test_forcefields/smirnoff99Frosst.offxml')

In [5]:
openmm_system = forcefield.create_openmm_system(topology)

In AmberToolsToolkitwrapper.computer_partial_charges_am1bcc: Molecule 'ethane' has more than one conformer, but this function will only generate charges for the first one.


In [24]:
## You can look at the XML serialised object
#pprint(XmlSerializer.serialize(openmm_system))

In [7]:
import parmed as pmd
pmd_lig_struct = pmd.openmm.load_topology(topology.to_openmm(), openmm_system, lig.conformers[0])

In [8]:
pmd_lig_struct

<Structure 8 atoms; 1 residues; 7 bonds; parametrized>

In [9]:
## only writing amber topologies files works with
pmd_lig_struct.save("ethane.prmtop", overwrite=True)
pmd_lig_struct.save("ethane.inpcrd", overwrite=True)

In [10]:
import BioSimSpace as BSS

/Users/toni_brain/miniconda3/envs/BSS/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::__1::vector<double, std::__1::allocator<double> > already registered; second conversion method ignored.
  return f(*args, **kwds)


In [11]:
#Reading in files to sire
ethane = BSS.IO.readMolecules(['ethane.prmtop','ethane.inpcrd'])

In [12]:
#Solvate
solv_ethane = BSS.Solvent.solvate("tip3p", molecule=ethane,
                                      box=3*[4*BSS.Units.Length.nanometer], work_dir='test')

In [17]:
#Save
BSS.IO.saveMolecules("solvated_ethane_smirnoff", solv_ethane, ["prm7", "gro87", "pdb"])

['/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/solvated_ethane_smirnoff.grotop',
 '/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/solvated_ethane_smirnoff.gro87',
 '/Users/toni_brain/Projects/People/Jeff_Wagner/bssopenffhack/solvated_ethane_smirnoff.pdb']

In [25]:
#This will not work because of non zero net charge works on command line using maxwarn 1 though.
protocol = BSS.Protocol.Minimisation()

In [ ]:
process = BSS.MD.run(solv_ethane, protocol, work_dir='test_min')